# Resources
https://github.com/ultimate010/crnn 
{Combination of Convolutional and Recurrent Neural Network for Sentiment Analysis of Short Texts}
https://keras.io/examples/imdb_bidirectional_lstm/

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
cd /content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation

/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation


# Imports Statements

In [ ]:
import collections
import pickle
import re
import random
import sys
import os 
import time


import gensim
from gensim.models.keyedvectors import KeyedVectors

from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict, KFold

from __future__ import print_function

import pandas as pd
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
from numpy import cumsum

import keras
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dropout, Activation, Flatten, \
    Embedding, Convolution1D, MaxPooling1D, AveragePooling1D, \
    Input, Dense, merge, Add,TimeDistributed, Bidirectional,SpatialDropout1D
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l2, l1_l2
from keras.constraints import maxnorm
from keras import callbacks
from keras.utils import generic_utils
from keras.models import Model
from keras.optimizers import Adadelta

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

# Load Data

In [ ]:
DataTagged = pd.read_csv("corpus/analyzed/lankadeepa_tagged_3.csv", ",")
# train_data, test_data = train_test_split(DataTagged, test_size=0.2, random_state=0)

In [ ]:
DataTagged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5010 entries, 0 to 5009
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   docid    5010 non-null   int64 
 1   comment  5010 non-null   object
 2   label    5010 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 117.5+ KB


In [ ]:
DataTagged

,docid,comment,label
0,1,නියම සිංහල මහත්මයෙක්,1
1,2,අන්න මිනිස්සු,1
2,3,ගොවි මහත්තයාගේ ගෙවත්තේ ලබු වැලේ ගෙඩි 50ක් පමණ ...,1
3,4,වන සිවුපාවන් වැනි මිනිසුන් මද දෙවි දේවතාවුන් ව...,1
4,5,අයියෝ සල්ලි,0
...,...,...,...
5005,5006,මේවා දේශපාලන පළිගැනීම් නොවේද?,0
5006,5007,මේ ආණ්ඩුව යහපාලනයක්ද යමපාලනයක්ද?? කෙරුවාව පළිග...,0
5007,5008,දූෂණ වංචා සම්බන්ධයෙන් චෝදනා ලැබූ පසුගිය ආණ්ඩුව...,0
5008,5009,වැරදිකාරයන්ට දඬුවම් කරන්න ඕනෑ,0


# preprocess Data

In [ ]:
# edit this later 
def text_preprocessing(train_data,test_data):
  train_data_texts = train_data['comment']
  train_data_labels = train_data['label']
  test_data_texts = test_data['comment']
  test_data_labels = test_data['label']


  comment_texts = []
  comment_labels = []

  train_text = []
  test_text = []
  train_labels=[]
  test_labels=[]

  for label in train_data_labels:
    if label == "POSITIVE":
      train_labels.append(1)
    else:
      train_labels.append(0)
  comment_labels.append(train_labels)

  for label in test_data_labels:
    if label == "POSITIVE":
      test_labels.append(1)
    else:
      test_labels.append(0)
  comment_labels.append(test_labels)
  

  for comment in train_data_texts:
    lines = []
    try:
      words = comment.split()
      lines += words
    except:
      continue
    train_text.append(lines)
  comment_texts.append(train_text)

  for comment in test_data_texts:
    lines = []
    try:
      words = comment.split()
      lines += words
    except:
      continue
    test_text.append(lines)
  comment_texts.append(test_text)


  return comment_texts,comment_labels

# edit this later 
def text_preprocessing_1(data):
  comments = data['comment']
  labels = data['label']

  comments_splitted = []
  labels_encoded = []

  for label in labels:
    if label == "POSITIVE":
      labels_encoded.append(1)
    else:
      labels_encoded.append(0)

  for comment in comments:
    lines = []
    try:
      words = comment.split()
      lines += words
    except:
      continue
    comments_splitted.append(lines)
  return comments_splitted,labels_encoded


def text_preprocessing_2(data):
  comments = data['comment']
  labels = data['label']

  comments_splitted = []

  for comment in comments:
    lines = []
    try:
      words = comment.split()
      lines += words
    except:
      continue
    comments_splitted.append(lines)

  return comments_splitted,labels

In [ ]:
comment_texts, comment_labels = text_preprocessing_2(DataTagged)

# prepare tokenizer

t = Tokenizer()
t.fit_on_texts(comment_texts)
vocab_size = len(t.word_index) + 1
print(vocab_size)

18352


In [ ]:
comment_texts[0]

['නියම', 'සිංහල', 'මහත්මයෙක්']

In [ ]:
encoded_docs = t.texts_to_sequences(comment_texts)
print(encoded_docs)

[[101, 131, 7049], [517, 25], [979, 2081, 7050, 1336, 7051, 7052, 7053, 1765, 4398, 6, 831, 7054, 7055, 7, 67, 86, 13], [189, 7056, 275, 132, 4399, 374, 4400, 275, 132, 150, 6, 7057, 7058, 1, 3194, 7059, 832, 7060, 3195, 7, 275, 16, 887], [419, 77], [30, 292, 286, 6, 7061, 980, 3196, 1528, 66, 66, 679, 17, 2495, 2496], [155, 58, 602, 226, 7062, 4401, 7063, 28, 345, 11, 2497], [7064, 264, 1337, 1336, 7065, 7066, 603, 329, 518, 150, 7067, 44, 173, 722, 19, 4402, 768, 1078, 4403, 4], [292, 723, 1336, 4404, 183], [2498, 981, 888, 393], [1336, 4404, 40, 72, 1766, 1336, 7068, 889, 2499, 2500], [1336, 7069, 72, 7070, 7071, 3197, 7072, 56, 110, 2501, 7073], [6, 7074, 22, 80, 7075, 1079, 7076, 1767, 74, 74, 1336, 1768, 4405, 890, 7077, 4405, 1336, 7078, 1080, 3198, 51, 184, 604, 14, 769], [7079, 1081], [568, 1082, 18, 80], [1190, 5, 16, 14, 770, 7080, 2502, 227, 132, 9, 4406, 72, 420, 78, 8, 469, 833, 3199, 219, 227, 68, 7081, 265, 44, 2082, 11, 78, 34, 2083, 421, 264, 602], [68, 1190, 605, 606

In [ ]:
encoded_docs[10]

[1336, 4404, 40, 72, 1766, 1336, 7068, 889, 2499, 2500]

In [ ]:
max_length = 155
padded_docs = pad_sequences(encoded_docs, maxlen=max_length)
print(padded_docs)

comment_labels = np.array(comment_labels)
padded_docs = np.array(padded_docs)

[[    0     0     0 ...   101   131  7049]
 [    0     0     0 ...     0   517    25]
 [    0     0     0 ...    67    86    13]
 ...
 [    0     0     0 ...   440   193  2195]
 [    0     0     0 ...   423    11    97]
 [    0     0     0 ...  1888   196 18351]]


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(padded_docs, comment_labels, test_size=0.2, random_state=0)
# X_train = np.array(X_train)
# X_test = np.array(X_test)
# y_train = np.array(y_train)
# y_test = np.array(y_test)
# comment_labels = np.array(comment_labels)

# Word Embedding

## Generate Embedding Metrix

In [ ]:
# keyed_fasttext_path = '/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/word_embedding/fasttext/fasttext_keyed_vectors/keyed.kv'
# word_vectors = KeyedVectors.load(keyed_fasttext_path, mmap='r')

In [ ]:
# embeddings_index = dict()
# for word, vocab_obj in word_vectors.vocab.items():
#   embeddings_index[word]=word_vectors[word]

In [ ]:
# create a weight matrix for words in training docs
# embedding_matrix = zeros((vocab_size, 300))
# for word, i in t.word_index.items():
# 	embedding_vector = embeddings_index.get(word)
# 	if embedding_vector is not None:
# 		embedding_matrix[i] = embedding_vector
# embedding_matrix_path = '/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/CNN RNN/fasttetx_embedding_matrix'
# pickle.dump(embedding_matrix, open(embedding_matrix_path, 'wb'))

## Load Embedding Matrix

In [ ]:
embedding_matrix_path = 'Sentiment Analysis/CNN RNN/fasttetx_embedding_matrix'
f = open(embedding_matrix_path, 'rb')
embedding_matrix= np.array(pickle.load(f))

# Models

## RNN(LSTM/GRU) model

In [ ]:
def build_RNN_model():
    main_input = Input(shape=(maxlen, ), dtype='int32', name='main_input')
    embedding  = Embedding(max_features, embedding_dims,
                  weights=[embedding_matrix], input_length=maxlen,
                  name='embedding' ,trainable=False)(main_input)

    embedding = Dropout(0.50)(embedding)

    x = RNN(rnn_output_size)(embedding)


    x = Dense(hidden_dims, activation='relu', init='he_normal',
              W_constraint = maxnorm(3), b_constraint=maxnorm(3),
              name='mlp')(x)

    x = Dropout(0.10, name='drop')(x)

    output = Dense(1, init='he_normal',
                   activation='sigmoid', name='output')(x)

    model = Model(input=main_input, output=output)
    model.compile(loss={'output':'binary_crossentropy'},
                optimizer=Adadelta(lr=0.95, epsilon=1e-06),
                metrics=["accuracy"])
    return model

## CNN+RNN(LSTM /GRU) model 

In [ ]:
def build_CNN_RNN_model():
    main_input = Input(shape=(maxlen, ), dtype='int32', name='main_input')
    embedding  = Embedding(max_features, embedding_dims,
                  weights=[embedding_matrix], input_length=maxlen,
                  name='embedding' ,trainable=False)(main_input)

    embedding = Dropout(0.50)(embedding)

    conv4 = Convolution1D(nb_filter=nb_filter,
                          filter_length=4,
                          border_mode='valid',
                          activation='relu',
                          subsample_length=1,
                          name='conv4')(embedding)
    maxConv4 = MaxPooling1D(pool_length=2,
                             name='maxConv4')(conv4)

    conv5 = Convolution1D(nb_filter=nb_filter,
                          filter_length=5,
                          border_mode='valid',
                          activation='relu',
                          subsample_length=1,
                          name='conv5')(embedding)
    maxConv5 = MaxPooling1D(pool_length=2,
                            name='maxConv5')(conv5)

    x = keras.layers.Concatenate(axis=1)([maxConv4, maxConv5])

    x = Dropout(0.15)(x)

    x = RNN(rnn_output_size)(x)


    x = Dense(hidden_dims, activation='relu', init='he_normal',
              W_constraint = maxnorm(3), b_constraint=maxnorm(3),
              name='mlp')(x)

    x = Dropout(0.10, name='drop')(x)

    output = Dense(1, init='he_normal',
                   activation='sigmoid', name='output')(x)

    model = Model(input=main_input, output=output)
    model.compile(loss={'output':'binary_crossentropy'},
                optimizer=Adadelta(lr=0.95, epsilon=1e-06),
                metrics=["accuracy"])
    return model

## CNN+BiLSTM

In [ ]:
def build_CNN_BiLSTM():
  # main model
  input = Input(shape=(maxlen,))
  embedding = Embedding(max_features,300,weights=[embedding_matrix],input_length=maxlen)(input)

  conv4 = Convolution1D(nb_filter=nb_filter,
                          filter_length=4,
                          border_mode='valid',
                          activation='relu',
                          subsample_length=1,
                          name='conv4')(embedding)
  maxConv4 = MaxPooling1D(pool_length=2,
                             name='maxConv4')(conv4)

  conv5 = Convolution1D(nb_filter=nb_filter,
                          filter_length=5,
                          border_mode='valid',
                          activation='relu',
                          subsample_length=1,
                          name='conv5')(embedding)
  maxConv5 = MaxPooling1D(pool_length=2,
                            name='maxConv5')(conv5)


  x = keras.layers.Concatenate(axis=1)([maxConv4, maxConv5])

  x = Dropout(0.15)(x)

  model =  Bidirectional (LSTM (300,return_sequences=True,dropout=0.8),merge_mode='concat')(x)
  model = TimeDistributed(Dense(300,activation='relu'))(model)
  model = Flatten()(model)
  model = Dense(300,activation='relu')(model)
  output = Dense(2,activation='softmax')(model)
  model = Model(input,output)
  model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
  return model


## BiLSTM 

In [ ]:
def build_BiLSTM_1_1():
  input = Input(shape=(maxlen,))
  embedding = Embedding(max_features,embedding_dims,weights=[embedding_matrix],input_length=maxlen)(input)

  model =  Bidirectional (LSTM (300,return_sequences=True,dropout=drop_out_value),merge_mode='concat')(embedding)
  model = TimeDistributed(Dense(300,activation='relu'))(model)
  model = Flatten()(model)
  model = Dense(300,activation='relu')(model)
  output = Dense(2,activation='softmax')(model)
  model = Model(input,output)
  model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
  return model

# final model
def build_BiLSTM_1_2():
  input = Input(shape=(maxlen,))
  embedding = Embedding(max_features,embedding_dims,weights=[embedding_matrix],input_length=maxlen)(input)

  model =  Bidirectional (LSTM (300,return_sequences=True,dropout=drop_out_value,kernel_regularizer=l2(0.01)),merge_mode='concat')(embedding)
  model = TimeDistributed(Dense(300,activation='relu'))(model)
  model = Flatten()(model)
  # model = Dense(300,activation='relu')(model)
  output = Dense(1,activation='sigmoid')(model)
  model = Model(input,output)
  model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
  return model

def build_BiLSTM_1_3():
  input = Input(shape=(maxlen,))
  embedding = Embedding(max_features,embedding_dims,weights=[embedding_matrix],input_length=maxlen)(input)

  model =  LSTM (512,return_sequences=True,dropout=drop_out_value,kernel_regularizer=l2(0.01))(embedding)
  model = LSTM (256,return_sequences=True,dropout=drop_out_value,kernel_regularizer=l2(0.01))(model)
  model = TimeDistributed(Dense(300,activation='relu'))(model)
  model = Flatten()(model)
  # model = Dense(300,activation='relu')(model)
  output = Dense(1,activation='sigmoid')(model)
  model = Model(input,output)
  model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
  return model

# BiLSTM with dropout regularization
def build_BiLSTM_2_1():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dims, input_length=maxlen,weights=[embedding_matrix]))
  model.add(Bidirectional(LSTM (300,dropout=drop_out_value),merge_mode='concat'))
  # model.add(Dropout(drop_out_value))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

# BiLSTM with l2 regularization
def build_BiLSTM_2_2():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dims, input_length=maxlen,weights=[embedding_matrix]))
  model.add(Bidirectional(LSTM (300,kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)),merge_mode='concat'))
  model.add(Dropout(drop_out_value))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model


# Train and Test

In [ ]:
def Train_Test_Model(model,X_train, X_test, y_train, y_test):

  print('Training and Testing...')
  test_accs = []
  first_run = True

  model = model
  if first_run:
      first_run = False
      print(model.summary())

  acc=[]
  val_acc=[]
  loss=[]
  val_loss=[]
  best_val_acc = 0
  best_test_acc = 0
  for j in range(nb_epoch):
      a = time.time()
      his = model.fit(X_train, y_train,
                      batch_size=batch_size,
                      validation_data=[X_test, y_test],
                      shuffle=True,
                      epochs=1, verbose=verbosity)
      acc+=his.history['accuracy']
      val_acc+=his.history['val_accuracy']
      loss+=his.history['loss']
      val_loss+=his.history['val_loss']
      # print('Epoch %d/%d\t%s' % (j + 1, nb_epoch, str(his.history)))
      if his.history['val_accuracy'][0] >= best_val_acc:
          score, test_acc = model.evaluate(X_test, y_test,
                                      batch_size=batch_size,
                                      verbose=2)
          best_val_acc = his.history['val_accuracy'][0]
          best_test_acc = test_acc
          print('Got best epoch  best val acc is %f test acc is %f' %
                (best_val_acc, best_test_acc))
          if len(test_accs) > 0:
              print('Current avg test acc:', str(np.mean(test_accs)))
      b = time.time()
      cost = b - a
      left = (nb_epoch - j - 1)
      print('One round cost %ds, %d round %ds %dmin left' % (cost, left,
                                                            cost * left,
                                                            cost * left / 60.0))
      test_accs.append(best_test_acc)
  print('Avg test acc:', str(np.mean(test_accs)))

# Cross Validation

In [ ]:
def Do_Cross_Validation(model,X,y):

  # Define per-fold score containers
  acc_per_fold = []
  loss_per_fold = []

  kfold = KFold(n_splits=folds, shuffle=True)

  fold_no = 1
  inputs = X
  targets = y
  for train, test in kfold.split(inputs, targets):
    model = model

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    his = model.fit(inputs[train], targets[train],
                  batch_size=batch_size,
                  shuffle=True,
                  epochs=nb_epoch, 
                  verbose=verbosity,
                  validation_split=validation_split)
    
    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1

  # == Provide average scores ==
  print('------------------------------------------------------------------------')
  print('Score per fold')
  for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
  print('------------------------------------------------------------------------')
  print('Average scores for all folds:')
  print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
  print(f'> Loss: {np.mean(loss_per_fold)}')
  print('------------------------------------------------------------------------')

# Plot Graphs

In [ ]:
def Plot_graphs():

  epochs=range(len(acc)) # Get number of epochs

  #------------------------------------------------
  # Plot training and validation accuracy per epoch
  #------------------------------------------------
  plt.plot(epochs, acc, 'r')
  plt.plot(epochs, val_acc, 'b')
  plt.title('Training and validation accuracy')
  plt.xlabel("Epochs")
  plt.ylabel("Accuracy")
  plt.legend(["Accuracy", "Validation Accuracy"])

  plt.figure()

  #------------------------------------------------
  # Plot training and validation loss per epoch
  #------------------------------------------------
  plt.plot(epochs, loss, 'r')
  plt.plot(epochs, val_loss, 'b')
  plt.title('Training and validation loss')
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.legend(["Loss", "Validation Loss"])

  plt.figure()


  # Expected Output
  # A chart where the validation loss does not increase sharply!

# Main

In [ ]:
batch_size = 32 # 64, 128
nb_filter = 200
filter_length = 4 # test with 2,3,4,5
hidden_dims = nb_filter * 2
nb_epoch = 2
RNN = LSTM 
rnn_output_size = 300 
folds = 3
maxlen = 155 #test with other values
max_features = embedding_matrix.shape[0] #vocab_size
embedding_dims = 300
drop_out_value = 0.5 #0.8 #0.3
verbosity = 1
validation_split = 0.2

In [ ]:
model = build_BiLSTM_1_2()
Train_Test_Model(model,X_train, X_test, y_train, y_test)
# Do_Cross_Validation(model,padded_docs,comment_labels)


Training and Testing...
Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 155)               0         
_________________________________________________________________
embedding_14 (Embedding)     (None, 155, 300)          5524800   
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 155, 600)          1442400   
_________________________________________________________________
time_distributed_14 (TimeDis (None, 155, 300)          180300    
_________________________________________________________________
flatten_14 (Flatten)         (None, 46500)             0         
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 46501     
Total params: 7,194,001
Trainable params: 7,194,001
Non-trainable params: 0
________________________

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4008 samples, validate on 1002 samples
Epoch 1/1
2496/4008 [=================>............] - ETA: 39s - loss: 2.9479 - accuracy: 0.6138

KeyboardInterrupt: ignored